Setup

In [154]:
import os
import requests
import json

from src.AvaGuardHelpers.avaguard_models import  User,  Observation


In [ ]:


####api_url = "https://api.avalancheguard.org:50001"
api_url = "http://127.0.0.1:8000"
loginForm = {"username": "", "password": ""}
endPoint = "token"
api_full_url = api_url + "/"+endPoint
response = requests.post(api_full_url, data=loginForm)
if response.status_code == 200:
    tokenDict = response.json()
else:
    tokenDict = {}


print(tokenDict)
#print(response.json())
print(response.status_code)

In [ ]:
head = {'Authorization': 'Bearer {}'.format(tokenDict["access_token"])}
print(head)
endPoint = "users/me"
api_full_url = api_url + "/"+endPoint
response = requests.get(api_full_url, headers=head)
if response.status_code == 200:
    user = response.json()
else:
    user = {}
print(user)


In [ ]:
head = {'Authorization': 'Bearer {}'.format(tokenDict["access_token"])}

endPoint = "observations"
api_full_url = api_url + "/"+endPoint
print(api_full_url)
response = requests.get(api_full_url, headers=head)
if response.status_code == 200:
    user = response.json()
else:
    user = {}
print(user)

### Load Data from Avalanche Center Manually Downloaded Json

In [ ]:
avalancheCenterJsonDir = "/Users/amiyaranjan/GitHub/w210/MIDS_DATASCI_210/data/ColoradoAvalancheCenter"
jsonFile = "Data_231101_240301.json"

#fullJsonFile = avalancheCenterJsonDir + "/"+jsonFile
fullJsonFile = os.path.join(avalancheCenterJsonDir,jsonFile)
print(fullJsonFile)

f = open(fullJsonFile)

jsonDataList = list(json.load(f))
print(str(jsonDataList)[:100])
#print(len(jsonDataList))
print("************")




In [ ]:
limiter = 2
for idx,jsonData in enumerate(jsonDataList):
    if idx > limiter:
        break
    if jsonData["avalanche_observations_count"] >0 and jsonData["assets"]:
        obs = Observation()
        obs.observation_id = jsonData["id"]
        if jsonData["saw_avalanche"] == "true" or jsonData["saw_avalanche"] == "True" :
            obs.avalanche_observed = True
        else:
            obs.avalanche_observed = False
        obs.avalanche_triggered = jsonData["triggered_avalanche"]
        obs.avalanche_experienced = jsonData["caught_in_avalanche"]
        obs.datetime_taken_UTC = str(jsonData["observed_at"])[:16]
        print(obs.datetime_taken_UTC)
        obs.imagefile_url = jsonData["assets"][0]["full_url"]
        obs.observation_notes = jsonData["description"]
        obs.lat = jsonData["latitude"]
        obs.lng = jsonData["longitude"]
        obs.general_location = f'Colorado/{jsonData["backcountry_zone"]["title"]}'
        obs.submitted_user="cskier"
        #print(obs.asdict())
        #print(dict(obs))
        print("************")
        api_url = "http://127.0.0.1:8000"
        endPoint = "observations/register_observation_json"
        api_full_url = api_url + "/"+endPoint
        obsjson = json.dumps(obs.asdict())
        #print(f"Now uploading ={obsjson}")
        obsrv = {}

        response = requests.post(api_full_url, obsjson, headers=head)
        print(response)
        if response.status_code == 200:
            obsrv = response.json()
        else:
            obsrv = {}

print(obsrv)

#print(f'idx={idx},data={jsonData}')
print("")
print("--------------------------")

